# Mnist Module

In [8]:
import numpy as np
import onnx
import os
import glob
from onnx import numpy_helper
import tvm
from tvm import te
import tvm.relay as relay

## Load module and param

In [9]:
mnist_model = onnx.load('../mnist/mnist-8.onnx')

In [10]:
input_name = "Input3"
shape_dict = {input_name: (1, 1, 28, 28)}
print(shape_dict)

{'Input3': (1, 1, 28, 28)}


In [11]:
mod, params = relay.frontend.from_onnx(mnist_model, shape_dict)

## Set Target 

In [12]:
# target_type = 'x86_64'
target_type = 'ARMv8'

In [13]:
if target_type == 'x86_64':
    target = tvm.target.Target('llvm')
    target_host = tvm.target.Target('llvm')
elif target_type == 'ARMv7':
    target = tvm.target.arm_cpu("rasp3b")
    target_host = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
elif target_type == 'ARMv8':
    target = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
    target_host = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
elif target_type == 'OpenCL':
    target = tvm.target.Target("opencl -device=intel_graphics")
    target_host = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
    
print(target)
print(target_host)

llvm -keys=arm_cpu,cpu -device=arm_cpu -link-params=0 -mattr=+neon -mtriple=aarch64-linux-gnu
llvm -keys=arm_cpu,cpu -device=arm_cpu -link-params=0 -mattr=+neon -mtriple=aarch64-linux-gnu


## Build

In [14]:
with tvm.transform.PassContext(opt_level=3):
# with relay.build_config(opt_level=3):    
    module_lib = relay.build(mod, target=target, target_host=target_host, params=params, mod_name='mnist')

In [15]:
print(module_lib)

## Export Lib

In [16]:
# lib export

In [17]:
if target_type == 'x86_64':
    lib_path   = "../module/x86_64/mnist_x86_64.so"
    param_path = "../module/x86_64/mnist_x86_64.params"
    module_lib.export_library(lib_path)
elif target_type == 'ARMv7':
    lib_path   = "../module/armv7/mnist_aarch64.so"
    param_path = "../module/armv7/mnist_aarch64.params"
    module_lib.export_library(lib_path, tvm.contrib.cc.cross_compiler('arm-linux-gnueabihf-g++'))
elif target_type == 'ARMv8':
    lib_path   = "../module/armv8/mnist_aarch64.so"
    param_path = "../module/armv8/mnist_aarch64.params"
    module_lib.export_library(lib_path, tvm.contrib.cc.cross_compiler('aarch64-linux-gnu-g++'))
elif target_type == 'OpenCL':
    lib_path   = "../module/opencl/mnist_opencl.so"
    param_path = "../module/opencl/mnist_opencl.params"
    module_lib.export_library(lib_path, tvm.contrib.cc.cross_compiler('aarch64-linux-gnu-g++'))    

##  Export Params

In [18]:
with open(param_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))

In [20]:
#module = tvm.contrib.graph_executor.GraphModule(module_lib["mnist"](ctx))